In [75]:
#pennylane circuit level
import pennylane as qml
from pennylane import numpy as np
from pennylane.qchem import molecular_hamiltonian
from pennylane import commutator
from scipy.linalg import expm
import matplotlib.pyplot as plt

# ========== 1. 分子设置 ==========
import pennylane as qml
from pennylane import numpy as np
from pennylane.qchem import molecule
from pennylane import commutator
import matplotlib.pyplot as plt

import pennylane as qml
from pennylane import numpy as np
from pennylane.qchem import molecule
from pennylane import commutator
import matplotlib.pyplot as plt

# 1. 设置分子结构
from pennylane.qchem import molecular_hamiltonian

from pyscf import gto, scf
from openfermionpyscf import run_pyscf
from openfermion.transforms import jordan_wigner
from openfermion.linalg import get_sparse_operator
import scipy

from openfermion import MolecularData
from openfermionpyscf import run_pyscf
from openfermion.transforms import jordan_wigner
from openfermion.linalg import get_sparse_operator

# Step 1: 构造 OpenFermion 分子对象（不要用 PySCF 的 gto.Mole）
geometry = [('H', (0.0, 0.0, 0.0)),
            ('H', (0.0, 0.0, 0.74))]  # 单位为 Å
basis = 'sto-3g'
multiplicity = 1
charge = 0

molecule = MolecularData(
    geometry=geometry,
    basis=basis,
    multiplicity=multiplicity,
    charge=charge
)

# Step 2: 运行 PySCF 计算（RHF + 哈密顿量）
molecule = run_pyscf(molecule, run_scf=True)

# Step 3: Jordan-Wigner 变换为 Qubit Hamiltonian
fermionic_hamiltonian = molecule.get_molecular_hamiltonian()
qubit_hamiltonian = jordan_wigner(fermionic_hamiltonian)

# Step 4: 转换为稀疏矩阵（如需矩阵形式演化）
from pennylane import PauliX, PauliY, PauliZ, Identity, Hamiltonian

def openfermion_to_qml_hamiltonian(qubit_op, n_qubits):
    coeffs = []
    ops = []
    for term, coef in qubit_op.terms.items():
        if len(term) == 0:
            ops.append(Identity(0))
        else:
            op_list = []
            for qubit_idx, pauli in term:
                if pauli == 'X':
                    op_list.append(PauliX(qubit_idx))
                elif pauli == 'Y':
                    op_list.append(PauliY(qubit_idx))
                elif pauli == 'Z':
                    op_list.append(PauliZ(qubit_idx))
            op = op_list[0] if len(op_list) == 1 else qml.prod(*op_list)
            ops.append(op)
        coeffs.append(np.real(coef))  # 只保留实部
    return Hamiltonian(coeffs, ops)

# 调用转换函数
n_qubits = molecule.n_qubits
cost_h = openfermion_to_qml_hamiltonian(qubit_hamiltonian, n_qubits)

def build_driver_h(n):
    return qml.Hamiltonian([1.0] * n, [qml.PauliX(i) for i in range(n)])

driver_h = build_driver_h(n_qubits)

def build_commutator_hamiltonian(H_d, H_c):
    comm_terms = []

    coeffs_d, ops_d = H_d.terms()
    coeffs_c, ops_c = H_c.terms()

    for c1, op1 in zip(coeffs_d, ops_d):
        for c2, op2 in zip(coeffs_c, ops_c):
            try:
                comm = commutator(op1, op2)
                # 如果 comm 是零算符（空 Sum），跳过
                if isinstance(comm, qml.ops.op_math.Sum) and len(comm.operands) == 0:
                    continue
                comm_terms.append(1j * c1 * c2 * comm)
            except Exception as e:
                continue

    return sum(comm_terms).simplify()


comm_h = build_commutator_hamiltonian(driver_h, cost_h)

print("driver_h is", driver_h)
print("cost_h is", cost_h)
print("commh_h is", comm_h)

from qiskit.quantum_info import SparsePauliOp
from pennylane.ops.op_math import Prod  # 用于多 Pauli 项的乘积

def qml_hamiltonian_to_sparse_pauli_op(qml_hamiltonian, n_qubits):
    coeffs = []
    paulis = []

    for coeff, op in zip(qml_hamiltonian.coeffs, qml_hamiltonian.ops):
        label = ['I'] * n_qubits

        # 多个 Pauli 相乘
        if isinstance(op, Prod):
            for o in op.operands:
                idx = o.wires[0]
                label[idx] = o.name[-1].upper()  # 确保是 'X', 'Y', 'Z'
        else:
            idx = op.wires[0]
            label[idx] = op.name[-1].upper()

        pauli_str = ''.join(label[::-1])  # little-endian 顺序
        paulis.append(pauli_str)
        coeffs.append(complex(coeff))

    return SparsePauliOp.from_list(list(zip(paulis, coeffs)))

cost_h_qiskit = qml_hamiltonian_to_sparse_pauli_op(cost_h, n_qubits)
driver_h_qiskit = qml_hamiltonian_to_sparse_pauli_op(driver_h, n_qubits)
comm_h_qiskit = qml_hamiltonian_to_sparse_pauli_op(comm_h, n_qubits)

print("driver_h is", driver_h_qiskit)
print("cost_h is", cost_h_qiskit)
print("commh_h is", comm_h_qiskit)

def falqon_layer(beta_k, cost_h, driver_h, delta_t, qc):
    for c, pauli_term in zip(cost_h.coeffs, cost_h.oplist):
        theta = 2 * c * delta_t
        apply_pauli_rotation(qc, theta, pauli_term)

    for c, pauli_term in zip(driver_h.coeffs, driver_h.oplist):
        theta = 2 * c * beta_k * delta_t
        apply_pauli_rotation(qc, theta, pauli_term)

def apply_pauli_rotation(qc, theta, pauli_term):
    wires = [int(w) for w in pauli_term.wires]
    pauli_str = pauli_term.name

    if pauli_str == 'X':
        qc.rx(theta, wires[0])
    elif pauli_str == 'Y':
        qc.ry(theta, wires[0])
    elif pauli_str == 'Z':
        qc.rz(theta, wires[0])
    elif pauli_str in ['XX', 'YY', 'ZZ']:
        q0, q1 = wires
        if pauli_str == 'XX':
            qc.h(q0)
            qc.h(q1)
        elif pauli_str == 'YY':
            qc.sdg(q0)
            qc.h(q0)
            qc.sdg(q1)
            qc.h(q1)
        elif pauli_str == 'ZZ':
            pass
        qc.cx(q0, q1)
        qc.rz(theta, q1)
        qc.cx(q0, q1)
        if pauli_str == 'XX':
            qc.h(q0)
            qc.h(q1)
        elif pauli_str == 'YY':
            qc.h(q0)
            qc.s(q0)
            qc.h(q1)
            qc.s(q1)
    else:
        raise NotImplementedError(f"Unsupported Pauli string: {pauli_str}")




driver_h is 1.0 * X(0) + 1.0 * X(1) + 1.0 * X(2) + 1.0 * X(3)
cost_h is -0.09706626816763125 * I(0) + 0.17141282644776926 * Z(0) + 0.1714128264477692 * Z(1) + -0.22343153690813433 * Z(2) + -0.22343153690813433 * Z(3) + 0.16868898170361205 * (Z(0) @ Z(1)) + 0.12062523483390411 * (Z(0) @ Z(2)) + 0.16592785033770338 * (Z(0) @ Z(3)) + 0.16592785033770338 * (Z(1) @ Z(2)) + 0.12062523483390411 * (Z(1) @ Z(3)) + 0.17441287612261575 * (Z(2) @ Z(3)) + -0.04530261550379926 * (X(0) @ X(1) @ Y(2) @ Y(3)) + 0.04530261550379926 * (X(0) @ Y(1) @ Y(2) @ X(3)) + 0.04530261550379926 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.04530261550379926 * (Y(0) @ Y(1) @ X(2) @ X(3))
commh_h is (0.3428256528955385+0j) * Y(0) + (0.3373779634072241+0j) * (Y(0) @ Z(1)) + (0.24125046966780822+0j) * (Y(0) @ Z(2)) + (0.33185570067540676+0j) * (Y(0) @ Z(3)) + (-0.09060523100759851+0j) * (Z(0) @ X(1) @ X(2) @ Y(3)) + (0.09060523100759851+0j) * (Z(0) @ Y(1) @ X(2) @ X(3)) + (0.3428256528955384+0j) * Y(1) + (0.3373779634072241+0j) *